### Digits worksheet

In [3]:
from typing import Iterable, Tuple
from collections import deque
from dataclasses import dataclass

PLUS, MINUS, TIMES, DIVIDE = "+", "-", "*", "/"
COMMUTATIVE = {PLUS: True, MINUS: False, TIMES: True, DIVIDE: False}
ACTIONS = [PLUS, MINUS, TIMES, DIVIDE]


def pairs(n: int, ordered: bool = False) -> Iterable[Tuple[int, int]]:
    for i in range(n-1):
        for j in range(i+1, n):
            yield (i, j)
            if not ordered:
                yield (j, i)

                
def use_values(i: int, j: int, values: list[int]) -> Tuple[int, int, list[int]]:
    i0, j0 = min(i, j), max(i, j)
    return values[i], values[j], values[:i0] + values[i0+1:j0] + values[j0+1:]
    
    
@dataclass
class State:
    operands: list[int]
    history: list[str]
    
    def __post_init__(self):
        self.operands.sort()
        self._hash = hash(tuple(self.operands))
        
    def __hash__(self):
        return self._hash
    
    def moves(self) -> Iterable["State"]:
        n = len(self.operands)
        for action in ACTIONS:
            ordered = COMMUTATIVE[action]
            for i, j in pairs(n, ordered):
                a, b, values = use_values(i, j, self.operands)
                if not (a and b):
                    continue
                if action == PLUS:
                    result = a + b
                elif action == MINUS:
                    result = a - b
                elif action == TIMES:
                    result = a * b
                elif action == DIVIDE:
                    if a % b:
                        continue
                    result = a // b

                operation = f"{a} {action} {b}"
                history = self.history.copy()
                history.append(operation)
                yield(State(values + [result], history))

In [7]:
target = 421
values = [5, 6, 10, 11, 18, 12]

# target = 113
# values = [6, 10, 11, 18]

init = State(list(values), [])
queue = deque([init])
seen = set([init])

count = 0
while queue:
    count += 1
    curr = queue.popleft()
    # print(f"\n{count:3d}: {str(curr)}")
    if target in curr.operands:
        print(curr)
        # break
        # print("...done.")
        continue
    for succ in curr.moves():
        if hash(succ) not in seen:
            # print(">>> " + str(succ))
            seen.add(hash(succ))
            queue.append(succ)

State(operands=[18, 421], history=['6 * 12', '10 + 72', '5 * 82', '11 + 410'])
State(operands=[6, 421], history=['10 * 12', '18 * 120', '2160 / 5', '432 - 11'])
State(operands=[421], history=['5 + 6', '11 + 12', '18 + 23', '10 * 41', '11 + 410'])
